In [1]:
import psutil
import pandas as pd
import os
import csv
import calendar
import datetime
import random
import numpy as np
os.getcwd()

#os.chdir('D:\Cincinnati\Classes\9. Graduate Case studies')
#os.getcwd()

'C:\\Users\\Jeevisha\\Desktop\\fifth third'

In [4]:
accts = pd.read_table('UC_CoBA_ACCT_BAL_MONTHLY_15.txt', usecols= ['I_REC_KEY','DATE_KEY'] ,sep="|")

C:\Users\Jeevisha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


In [5]:
accts.DATE_KEY.unique()

array(['31OCT2018', '30NOV2018', '31DEC2018', '31JAN2019', '28FEB2019',
       '31MAR2019', '30APR2019', '31MAY2019', '31JUL2019', '31AUG2019',
       '30SEP2019', '31OCT2019', '30NOV2019', '31DEC2019'], dtype=object)

In [27]:
#keys = pd.read_csv('final_filtered_keys.csv')

In [28]:
#table = pd.merge(accts,keys)

In [31]:
#masterlist = table.I_REC_KEY.unique().tolist()

In [6]:
date_format = "%d%b%Y"

In [7]:
attrition_flag = pd.read_csv('attritionflags.csv')

In [8]:
# Load accts and attrition flag file before 

def target_func2(for_month,months,rate):         ### rate =1 for target rate , rate anything other than 1 will output the file
    acct = accts[accts['DATE_KEY']== for_month]
    
    for_date = datetime.datetime.strptime(for_month, date_format)
    month = for_date.month
    file_month = month
    year = for_date.year + month // 12
    month = month % 12 + 1
    day = max(1, calendar.monthrange(year,month)[1])
    
    last_month = datetime.datetime(year,month,day,0,0,0)
    last_month = last_month.strftime('%d%b%Y').upper()
    
    attrition = attrition_flag[attrition_flag['attritionflag']==1]
    attrition = attrition[attrition['lastdate']==last_month]
    file = pd.merge(acct,
                   attrition,
                   left_on = 'I_REC_KEY',
                   right_on = 'I_REC_KEY',
                   how = 'left')
    target = [] 
    for value in file["attritionflag"]: 
        if value == 1: 
            target.append(1) 
        else: 
            target.append(0) 
    file['target']=target
    file['month']=file_month
    file = file.drop(columns=['lastdate','lastdateindate','attritionflag'],axis=1)
    
    target_rate = file[file['target']==1].shape[0]/file.shape[0]
    if rate == 1: 
        return target_rate 
    else: 
        return file

In [9]:
nov_file = target_func2('30NOV2018',1,0)

In [10]:
dec_file = target_func2('31DEC2018',1,0)

In [11]:
jan_file = target_func2('31JAN2019',1,0)

In [12]:
feb_file = target_func2('28FEB2019',1,0)

In [13]:
mar_file = target_func2('31MAR2019',1,0)

In [14]:
apr_file = target_func2('30APR2019',1,0)

In [15]:
may_file = target_func2('31MAY2019',1,0)

In [16]:
jun_file = target_func2('30JUN2019',1,0)

In [17]:
jul_file = target_func2('31JUL2019',1,0)

In [18]:
aug_file = target_func2('31AUG2019',1,0)

In [19]:
sep_file = target_func2('30SEP2019',1,0)

In [20]:
oct_file = target_func2('31OCT2019',1,0)

In [21]:
df = nov_file.append([dec_file, jan_file,feb_file,mar_file,apr_file,may_file,jun_file,jul_file,aug_file,sep_file,oct_file])

In [36]:
df.shape
df.head()

,I_REC_KEY,DATE_KEY,target,month
0,8923024,30NOV2018,0,11
1,8911235,30NOV2018,0,11
2,8908665,30NOV2018,0,11
3,10856643,30NOV2018,0,11
4,9697506,30NOV2018,0,11


In [46]:
final_df = pd.DataFrame(columns=['I_REC_KEY','DATE_KEY','target','month'])

for i in range(1,12):
    if i == 1:
        sam_key = (df[df['month']==i]).sample(7000)
        final_df = final_df.append(sam_key)
    else:
        df_temp = df[df['month']==i]
        sam_key = df_temp[~df_temp.I_REC_KEY.isin(final_df['I_REC_KEY'])].sample(7000)
        final_df = final_df.append(sam_key)

In [47]:
final_df.shape

(77000, 4)

In [48]:
final_df.I_REC_KEY.nunique()

77000

In [49]:
final_df.to_csv('7000_permonth_accts.csv')

## Working Code Till Here

In [26]:
sam_key = df.sample(5000)

In [27]:
sam_key.shape

(5000, 4)

In [28]:
sam_key

,I_REC_KEY,DATE_KEY,target,month
2362808,127817466,31JAN2019,0,1
1713433,98791163,31MAR2019,0,3
143103,8473931,31DEC2018,0,12
862681,41867738,31JAN2019,0,1
1363640,77928956,31DEC2018,0,12
3043214,149860372,31DEC2018,0,12
484801,14337842,30JUN2019,0,6
1491024,87827200,30JUN2019,1,6
3150402,153347877,30JUN2019,0,6
2761100,145760164,31MAR2019,0,3


In [24]:
df.to_csv('Monthly_acct_with_target.txt',sep='|',index=None)

KeyboardInterrupt: 

In [ ]:
df_test = pd.read_csv('Monthly_acct_with_target.csv')

In [103]:
df['month'].value_counts()

10    3300954
9     3298991
8     3292190
7     3277657
6     3268563
5     3260230
4     3096758
3     3086812
2     3078313
1     3069374
12    3067238
11    3065941
Name: month, dtype: int64

In [105]:
df.I_REC_KEY.nunique()

3750397

In [138]:
df_uniq=df.copy()
df_uniq = df_uniq.drop_duplicates(['I_REC_KEY'])

In [139]:
df_uniq.shape

(3750397, 4)

In [140]:
df_uniq.head()

,I_REC_KEY,DATE_KEY,target,month
0,8923024,30NOV2018,0,11
1,8911235,30NOV2018,0,11
2,8908665,30NOV2018,0,11
3,10856643,30NOV2018,0,11
4,9697506,30NOV2018,0,11


In [192]:
values = df['month'].unique().tolist()

In [193]:
values

[11, 12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [114]:
#container = [
 #   random.sample(masterlist, 5000)
  #  for l in values
#]

In [196]:
cont = pd.DataFrame(columns=['I_REC_KEY'])
cont

,I_REC_KEY


In [201]:
for l in range(1,12):
    masterlist_month = df_uniq[df_uniq['month'] == l]
    masterlist_month = masterlist_month.iloc[:,0].tolist()
    cont['I_REC_KEY'] = random.sample(masterlist_month,7000)
    

ValueError: Length of values does not match length of index

In [198]:
cont

,I_REC_KEY
0,94623894
1,138792633
2,99718841
3,123594407
4,12993046
5,149020559
6,45012581
7,127438941
8,145904944
9,114644407


In [120]:
check = pd.merge(container[0],df_uniq,on='I_REC_KEY',how='left')

ValueError: can not merge DataFrame with instance of type <class 'list'>

In [67]:
calendar.monthrange(2018,12)

(5, 31)

In [65]:
target_rate = []
month_list = []
final_df = pd.DataFrame()
months_before = 2          ### months before you want to capture attrition 
start_date_c = '31NOV2018' ### change the month you want to start with

In [66]:
# Loop for all months
for i in range(accts.DATE_KEY.unique().shape[0]-months_before):
    final_df.append(target_func2(start_date_c,months_before-1,0))  
    #month_list.append(start_date_c[2:])
    start_date = datetime.datetime.strptime(start_date_c, date_format)
    month = start_date.month
    year = start_date.year + month // 12
    month = month % 12 + 1
    day = max(1, calendar.monthrange(year,month)[1])
    start_date_c = datetime.datetime(year,month,day,0,0,0)
    start_date_c = start_date_c.strftime('%d%b%Y').upper()

ValueError: day is out of range for month

In [ ]:
target_file = pd.DataFrame({'Month':month_list,'Target Rate':target_rate})
target_file

In [3]:
tran = pd.read_table('UC_CoBA_TRAN_SUMM_MONTHLY_15.txt',sep='|',nrows=10)

In [4]:
tran.columns.values

array(['ACCT_KEY', 'I_REC_KEY', 'DATE_KEY', 'ACH_IN_MTD_AMT',
       'ACH_IN_MTD_QTY', 'ACH_OUT_MTD_AMT', 'ACH_OUT_MTD_QTY',
       'CHECK_CASHING_MTD_AMT', 'CHECK_CASHING_MTD_QTY',
       'CHK_WRITTEN_MTD_AMT', 'CHK_WRITTEN_MTD_QTY', 'DEBIT_CARD_MTD_AMT',
       'DEBIT_CARD_MTD_QTY', 'MOBILE_IMD_DEP_AMT', 'MOBILE_IMD_DEP_QTY',
       'MOBILE_STD_DEP_AMT', 'MOBILE_STD_DEP_QTY'], dtype=object)

In [5]:
tran_ach = pd.read_table('UC_CoBA_TRAN_SUMM_MONTHLY_15.txt',sep='|',usecols = ['ACCT_KEY', 'I_REC_KEY', 'DATE_KEY', 'ACH_IN_MTD_AMT',
       'ACH_IN_MTD_QTY', 'ACH_OUT_MTD_AMT', 'ACH_OUT_MTD_QTY'] )

In [6]:
tran_ach.shape

(42624951, 7)

In [7]:
tran_ach.head(10)

,ACCT_KEY,I_REC_KEY,DATE_KEY,ACH_IN_MTD_AMT,ACH_IN_MTD_QTY,ACH_OUT_MTD_AMT,ACH_OUT_MTD_QTY
0,3284078,9666410,31OCT2018,3409.27,2,0.00,0
1,3683895,9023693,31OCT2018,5392.35,2,352.61,4
2,2223868,9035845,31OCT2018,6205.11,4,67.93,2
3,3400641,9676268,31OCT2018,0.00,0,4205.93,4
4,2599780,9350982,31OCT2018,5130.46,3,8341.19,17
5,3624223,9036014,31OCT2018,1517.64,2,6.00,1
6,3652861,9352007,31OCT2018,4654.04,3,1487.29,30
7,2685663,9345326,31OCT2018,4723.22,3,458.03,5
8,2460866,9040993,31OCT2018,4648.60,2,894.42,3
9,2117565,9334400,31OCT2018,2182.00,1,404.52,2


In [13]:
acc = pd.read_table('UC_CoBA_ACCT_BAL_MONTHLY_15.txt',sep="|",nrows=10000)

In [12]:
acc.columns.values

array(['I_REC_KEY', 'DATE_KEY', 'AVG_MONTHLY_BAL_AMT',
       'LAST_STMT_BAL_AMT', 'MRKTG_PROD_NAME'], dtype=object)

In [15]:
acc.shape

(10000, 5)

In [16]:
acc.head(50)

,I_REC_KEY,DATE_KEY,AVG_MONTHLY_BAL_AMT,LAST_STMT_BAL_AMT,MRKTG_PROD_NAME
0,8923024,31OCT2018,2653.0,2487.0,REGULAR BUS CHECKING
1,8923024,31DEC2018,2794.0,4160.0,REGULAR BUS CHECKING
2,8923024,30NOV2018,2578.0,3626.0,REGULAR BUS CHECKING
3,8923024,31JAN2019,2332.0,2374.0,REGULAR BUS CHECKING
4,8923024,28FEB2019,2357.0,2648.0,REGULAR BUS CHECKING
5,8923024,30NOV2019,5175.0,3004.0,REGULAR BUS CHECKING
6,8923024,31MAR2019,3485.0,2116.0,REGULAR BUS CHECKING
7,8923024,30APR2019,2409.0,4135.0,REGULAR BUS CHECKING
8,8923024,31MAY2019,2008.0,3551.0,REGULAR BUS CHECKING
9,8923024,30JUN2019,2404.0,1774.0,REGULAR BUS CHECKING


In [17]:
hh_sum = pd.read_table('UC_CoBA_HH_SUMM_MONTHLY_15.txt',sep='|',nrows=10000)

In [19]:
hh_sum.columns.values

array(['HH_KEY', 'DATE_KEY', 'CONS_LOAN_ACCT_QTY', 'CONS_LOAN_BAL_AMT',
       'CONS_LOAN_WAR_PCT', 'CONS_LOAN_IND', 'CONS_DEPOSIT_ACCT_QTY',
       'CONS_DEPOSIT_BAL_AMT', 'CONS_DEPOSIT_IND', 'MORTGAGE_ACCT_QTY',
       'MORTGAGE_BAL_AMT', 'MORTGAGE_IND', 'SAFEBOX_ACCT_QTY',
       'SAFEBOX_BAL_AMT', 'SAFEBOX_IND', 'SAVINGS_ACCT_QTY',
       'SAVINGS_BAL_AMT', 'SAVINGS_IND', 'CD_ACCT_QTY', 'CD_BAL_AMT',
       'CD_IND', 'CHECKING_ACCT_QTY', 'CHECKING_BAL_AMT',
       'CREDIT_CARD_ACCT_QTY', 'CREDIT_CARD_BAL_AMT', 'CREDIT_CARD_IND',
       'DIRECT_LOAN_ACCT_QTY', 'DIRECT_LOAN_BAL_AMT', 'DIRECT_LOAN_IND',
       'HOME_EQUITY_ACCT_QTY', 'HOME_EQUITY_BAL_AMT', 'HOME_EQUITY_IND',
       'DEBIT_CARD_ACCT_QTY', 'DEBIT_CARD_IND', 'OLDEST_OPEN_DATE',
       'BRANCH_NBR', 'AFFIL_NBR', 'HH_TYPE_CODE', 'HH_SEG_CODE',
       'DIRECT_DEP_IND', 'LAST_DIRECT_DEPOSIT_AMT', 'ID_ALERT_IND',
       'IXI_CLUSTER_CODE', 'ACTIVE_CHK_IND', 'HABITUAL_OD_IND',
       'ID_ALERT_TYPE_CODE', 'GPR_IND', 'GPR_DIREC

In [24]:
hh_sum.iloc[:,10:20].head(10) # CONS_LOAN_IND, CONS_DEPOSIT_IND

,MORTGAGE_BAL_AMT,MORTGAGE_IND,SAFEBOX_ACCT_QTY,SAFEBOX_BAL_AMT,SAFEBOX_IND,SAVINGS_ACCT_QTY,SAVINGS_BAL_AMT,SAVINGS_IND,CD_ACCT_QTY,CD_BAL_AMT
0,0,N,0,0,N,0,0,N,0,0
1,0,N,0,0,N,2,274465,Y,0,0
2,713344,Y,0,0,N,0,0,N,0,0
3,0,N,0,0,N,0,0,N,0,0
4,0,N,0,0,N,0,0,N,0,0
5,0,N,0,0,N,0,0,N,0,0
6,0,N,0,0,N,0,0,N,0,0
7,0,N,0,0,N,0,0,N,0,0
8,0,N,0,0,N,0,0,N,0,0
9,0,N,0,0,N,1,1613,Y,0,0
